# ETL: автоматизация подготовки данных семинары

## Урок 3. Получение денормализованных таблиц из нормализованных

1. Денормализуйте таблицу так, чтобы не нужно было для каждого рекламодателя постоянно подсчитывать количество кампаний и продаж.

In [1]:
import init_spark_env